
# AWS Lab Exercise — IAM Groups, Users, and Custom Policies for EC2 Access Control (CLI Based)

### Objective
This lab continues from the previous **Console-based IAM exercise**.  
You will now perform the same scenario using **AWS CLI**, where the CLI admin user (`cli-admin`) has already been created and assigned **AdministratorAccess** via an admin group using the AWS Console.

---

### Precondition (Console Step)
Before starting the CLI section, perform this one-time setup in the AWS Management Console:

1. Go to **IAM → User groups → Create group**
   - **Group name:** `cli-admin-group`
   - Attach policy: `AdministratorAccess`
   - Click **Create group**

2. Go to **IAM → Users → Create user**
   - **User name:** `cli-admin`
   - Enable **Access key - Programmatic access**
   - Skip permission assignment
   - Click **Create user**

3. Add the user to the group:
   - **IAM → Users → cli-admin → Groups → Add user to groups → cli-admin-group**

4. Download the **Access key ID** and **Secret access key** for `cli-admin`.

Now continue the rest of the lab using AWS CLI.

---



## Step 1: Configure and Verify CLI Admin Profile

### Configure AWS CLI
```bash
aws configure --profile cli-admin
```
Enter:
- **AWS Access Key ID:** `<copied-access-key>`  
- **AWS Secret Access Key:** `<copied-secret-key>`  
- **Default region name:** `us-east-1`  
- **Default output format:** `json`  

### Verify Configuration
```bash
aws sts get-caller-identity --profile cli-admin
```
You should see the ARN of `cli-admin` confirming CLI access is configured successfully.



## Step 2: Create IAM Groups and Users for CLI Access

### Create Groups
```bash
aws iam create-group --group-name developer_cli --profile cli-admin
```

### Attach Managed Policies
```bash
aws iam attach-group-policy --group-name developer_cli --policy-arn arn:aws:iam::aws:policy/AmazonEC2FullAccess --profile cli-admin
```

### Create Users
```bash
aws iam create-user --user-name dev11 --profile cli-admin
```

### Add Users to Groups
```bash
aws iam add-user-to-group --user-name dev11 --group-name developer_cli --profile cli-admin
```

### Verification
```bash
aws iam list-users --profile cli-admin
aws iam list-groups --profile cli-admin
aws iam list-groups-for-user --user-name dev11 --profile cli-admin
```



## Step 3: Create Access Keys and Configure CLI for dev11 and dev12

### Create Access Keys
```bash
aws iam create-access-key --user-name dev11 --profile cli-admin
```
Copy and securely store both **Access Key IDs** and **Secret Access Keys**.

## Launch another terminal

### Configure CLI Profiles for Each User
```bash
aws configure --profile dev11
```
Enter respective credentials and region `us-east-1`.

### Verify Setup
```bash
aws sts get-caller-identity --profile dev11
```



## Step 4: Launch and Manage EC2 Instances Using dev11

### Launch EC2 Instance
```bash
aws ec2 run-instances   --image-id ami-052064a798f08f0d3  --count 1   --instance-type t2.micro   --key-name EC2-A   --tag-specifications "ResourceType=instance,Tags=[{Key=Name,Value=CLI-Instance-1}]"   --region us-east-1   --profile dev11
```

### Verify Instance Creation
```bash
aws ec2 describe-instances --filters "Name=tag:Name,Values=CLI-Instance-1" --region us-east-1 --profile dev11
```

### Stop and Start Instance
```bash
aws ec2 stop-instances --instance-ids $INSTANCE_ID --region us-east-1 --profile dev11
aws ec2 start-instances --instance-ids $INSTANCE_ID --region us-east-1 --profile dev11
```

### Terminate Instance (should succeed initially)
```bash
aws ec2 terminate-instances --instance-ids $INSTANCE_ID --region ap-south-1 --profile dev11
```



---

## Reflection

✅ **Key Learnings**
- The `cli-admin` user inherits administrative privileges via its group (best practice).  
- IAM users and groups can be fully managed via CLI.  
- Managed policies provide broad access, while custom policies restrict specific actions.  
- Explicit *Deny* overrides *Allow*, enforcing least privilege even on otherwise powerful roles.  


---
